In [25]:
#from google.colab import files, drive
#drive.mount('/content/gdrive')
# files.upload()

In [42]:
!pip install transformers
!pip install silence_tensorflow

In [43]:
%matplotlib inline
#Importing Transformers
import transformers
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer

#Importing Pandas
import pandas as pd

#Importing Numpy
import numpy as np

from silence_tensorflow import silence_tensorflow
silence_tensorflow()
import torch
import json
import nltk
import re
import string
import matplotlib.pyplot as plt
import operator

In [44]:
config = AutoConfig.from_pretrained("ProsusAI/finbert")
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [46]:
SBI_news = pd.read_excel('articles_sbi (3).xlsx')
SBI_news = SBI_news.loc[:, ~SBI_news.columns.str.contains('^Unnamed')]

In [47]:
SBI_news.head()

,Date,Media,Title,Article,Summary,Classifier
0,"Mar 8, 2020",Business Standard,"SBI says YES Bank survival crucial, can invest...","Dear Reader,\n\nBusiness Standard has always s...",As we battle the economic impact of the pandem...,NaN
1,"Mar 5, 2020",Mint,Yes Bank will not be nationalised; SBI board m...,Yes Bank Ltd will not be nationalised under th...,“State Bank will only hold a stake in Yes Bank...,NaN
2,"Mar 6, 2020",Business Today,"SBI, LIC to rescue Yes Bank? Here's what we kn...",The government has knocked on the doors of the...,The government has knocked on the doors of the...,NaN
3,"Mar 13, 2020",Mint,"SBI, LIC & top private banks together to infus...",Mumbai: State bank of India has submitted a pr...,"According to the proposal, SBI will infuse ₹72...",NaN
4,"Mar 13, 2020",The Economic Times,"YES Bank bailout plan: SBI, ICICI, Jhunjhunwal...",Mumbai/New Delhi: The Cabinet approved the rec...,Other participants in the plan include Housing...,NaN


In [50]:
SBI_news_mod.head()

,Date,Media,Title,Article,Summary,Classifier
0,"Mar 8, 2020",Business Standard,"SBI says YES Bank survival crucial, can invest...","Dear Reader,\n\nBusiness Standard has always s...",As we battle the economic impact of the pandem...,NaN
1,"Mar 5, 2020",Mint,Yes Bank will not be nationalised; SBI board m...,Yes Bank Ltd will not be nationalised under th...,“State Bank will only hold a stake in Yes Bank...,NaN
2,"Mar 6, 2020",Business Today,"SBI, LIC to rescue Yes Bank? Here's what we kn...",The government has knocked on the doors of the...,The government has knocked on the doors of the...,NaN
3,"Mar 13, 2020",Mint,"SBI, LIC & top private banks together to infus...",Mumbai: State bank of India has submitted a pr...,"According to the proposal, SBI will infuse ₹72...",NaN
4,"Mar 13, 2020",The Economic Times,"YES Bank bailout plan: SBI, ICICI, Jhunjhunwal...",Mumbai/New Delhi: The Cabinet approved the rec...,Other participants in the plan include Housing...,NaN


In [49]:
SBI_news_mod = SBI_news.copy(deep=True)

In [51]:
classes=['positive','negative','neutral']

In [54]:
SBI_news_mod.drop('Classifier',inplace=True, axis=1)
SBI_news_mod.head()

,Date,Media,Title,Article,Summary
0,"Mar 8, 2020",Business Standard,"SBI says YES Bank survival crucial, can invest...","Dear Reader,\n\nBusiness Standard has always s...",As we battle the economic impact of the pandem...
1,"Mar 5, 2020",Mint,Yes Bank will not be nationalised; SBI board m...,Yes Bank Ltd will not be nationalised under th...,“State Bank will only hold a stake in Yes Bank...
2,"Mar 6, 2020",Business Today,"SBI, LIC to rescue Yes Bank? Here's what we kn...",The government has knocked on the doors of the...,The government has knocked on the doors of the...
3,"Mar 13, 2020",Mint,"SBI, LIC & top private banks together to infus...",Mumbai: State bank of India has submitted a pr...,"According to the proposal, SBI will infuse ₹72..."
4,"Mar 13, 2020",The Economic Times,"YES Bank bailout plan: SBI, ICICI, Jhunjhunwal...",Mumbai/New Delhi: The Cabinet approved the rec...,Other participants in the plan include Housing...


In [55]:
SBI_news_mod = SBI_news_mod.dropna()

In [56]:
SBI_news_mod['title_encoded'] = SBI_news_mod['Title'].apply(lambda x: tokenizer.encode_plus(x,
                                                                                            truncation=True,
                                                                                            padding=True,
                                                                                            return_tensors="pt"))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [57]:
SBI_news_mod['sentiment'] = SBI_news_mod['title_encoded'].apply(lambda x: dict(zip(classes, torch.softmax(model(**x)[0], dim=1).tolist()[0])))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [58]:
SBI_news_mod.head()

,Date,Media,Title,Article,Summary,title_encoded,sentiment
0,"Mar 8, 2020",Business Standard,"SBI says YES Bank survival crucial, can invest...","Dear Reader,\n\nBusiness Standard has always s...",As we battle the economic impact of the pandem...,"[input_ids, token_type_ids, attention_mask]","{'positive': 0.4953855276107788, 'negative': 0..."
1,"Mar 5, 2020",Mint,Yes Bank will not be nationalised; SBI board m...,Yes Bank Ltd will not be nationalised under th...,“State Bank will only hold a stake in Yes Bank...,"[input_ids, token_type_ids, attention_mask]","{'positive': 0.02600766532123089, 'negative': ..."
2,"Mar 6, 2020",Business Today,"SBI, LIC to rescue Yes Bank? Here's what we kn...",The government has knocked on the doors of the...,The government has knocked on the doors of the...,"[input_ids, token_type_ids, attention_mask]","{'positive': 0.05002627521753311, 'negative': ..."
3,"Mar 13, 2020",Mint,"SBI, LIC & top private banks together to infus...",Mumbai: State bank of India has submitted a pr...,"According to the proposal, SBI will infuse ₹72...","[input_ids, token_type_ids, attention_mask]","{'positive': 0.5257067680358887, 'negative': 0..."
4,"Mar 13, 2020",The Economic Times,"YES Bank bailout plan: SBI, ICICI, Jhunjhunwal...",Mumbai/New Delhi: The Cabinet approved the rec...,Other participants in the plan include Housing...,"[input_ids, token_type_ids, attention_mask]","{'positive': 0.6517215967178345, 'negative': 0..."


In [59]:
SBI_news_mod['sentiment_news'] = SBI_news_mod['sentiment'].apply(lambda x: max(x.items(), key=operator.itemgetter(1))[0])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [60]:
SBI_news_mod.head()

,Date,Media,Title,Article,Summary,title_encoded,sentiment,sentiment_news
0,"Mar 8, 2020",Business Standard,"SBI says YES Bank survival crucial, can invest...","Dear Reader,\n\nBusiness Standard has always s...",As we battle the economic impact of the pandem...,"[input_ids, token_type_ids, attention_mask]","{'positive': 0.4953855276107788, 'negative': 0...",neutral
1,"Mar 5, 2020",Mint,Yes Bank will not be nationalised; SBI board m...,Yes Bank Ltd will not be nationalised under th...,“State Bank will only hold a stake in Yes Bank...,"[input_ids, token_type_ids, attention_mask]","{'positive': 0.02600766532123089, 'negative': ...",neutral
2,"Mar 6, 2020",Business Today,"SBI, LIC to rescue Yes Bank? Here's what we kn...",The government has knocked on the doors of the...,The government has knocked on the doors of the...,"[input_ids, token_type_ids, attention_mask]","{'positive': 0.05002627521753311, 'negative': ...",neutral
3,"Mar 13, 2020",Mint,"SBI, LIC & top private banks together to infus...",Mumbai: State bank of India has submitted a pr...,"According to the proposal, SBI will infuse ₹72...","[input_ids, token_type_ids, attention_mask]","{'positive': 0.5257067680358887, 'negative': 0...",positive
4,"Mar 13, 2020",The Economic Times,"YES Bank bailout plan: SBI, ICICI, Jhunjhunwal...",Mumbai/New Delhi: The Cabinet approved the rec...,Other participants in the plan include Housing...,"[input_ids, token_type_ids, attention_mask]","{'positive': 0.6517215967178345, 'negative': 0...",positive


In [61]:
SBI_news_mod['sentiment_news'].value_counts()

neutral     113
negative     41
positive     31
Name: sentiment_news, dtype: int64

In [62]:
SBI_news_mod.to_excel("sbi_sentiment2.xlsx")